In [1]:
import sys
import os

# Only add the project root to sys.path
root_path = os.path.abspath('..')
if root_path not in sys.path:
    sys.path.insert(0, root_path)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from Functions.metrics import f_measure, covering
from Functions.true_cps import read_true_change_points
from Algorithms.floss import floss
# from coverage import calculate_coverage_score

# Define the dataset name
dataset_name = 'CricketX'

# Path to the dataset and description files
data_path = f'..\Data\{dataset_name}.txt'
desc_path = '..\Data\TrueCPS\desc.txt'

# Load the dataset
data = np.loadtxt(data_path)

# Read true change points
true_change_points = read_true_change_points(desc_path, dataset_name)

cps = floss(data_path, 200, 50, 5)

print(f"Detected Change Points by Binseg: {detected_cps}")
print(f"True Change Points: {true_cps}")
print(f"F1 Score: {f1_score}")
print(f"Covering Score: {covering_score}")

# Visualization
plt.figure(figsize=(15, 6))  # Adjust the figure size to make it wider
plt.plot(data, label='Time Series Data')

# Draw vertical lines for each change point detected by binseg
for cp in change_points:
    plt.axvline(x=cp, color='red', linestyle='--', linewidth=1, label='Detected Change Point' if cp == change_points[0] else "")

# Draw vertical lines for each true change point
for cp in true_change_points:
    plt.axvline(x=cp, color='green', linestyle='-', linewidth=1, label='True Change Point' if cp == true_change_points[0] else "")

plt.xlabel('Time')
plt.ylabel('Value')
plt.title(f'Time Series Data of {dataset_name} with Detected and True Change Points')
plt.legend()
plt.show()

AttributeError: 'str' object has no attribute 'copy'